In [1]:
!pip install numpy nltk transformers torch PyMuPDF streamlit
!pip install --upgrade protobuf  # Fix for potential protobuf version issues

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 10.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux201

In [2]:
import fitz  # PyMuPDF
import re
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
from transformers import BertTokenizer, BertModel, pipeline
import torch
import streamlit as st

# Download necessary NLTK data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    return text

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = vector
    return embeddings_index

def get_embedding(word, embeddings_index):
    return embeddings_index.get(word, np.zeros(50))

class ContextualEmbedder:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')

    def get_contextual_embeddings(self, text):
        inputs = self.tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
        outputs = self.model(**inputs)
        return outputs.last_hidden_state

def create_qa_pipeline():
    qa_pipeline = pipeline('question-answering', model='bert-large-uncased-whole-word-masking-finetuned-squad')
    return qa_pipeline

def answer_question(question, context, qa_pipeline):
    result = qa_pipeline(question=question, context=context)
    return result['answer']

In [4]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    pdf_path = filename
    break

# Extract text from uploaded PDF
text = extract_text_from_pdf(pdf_path)

# Display extracted text (first 1000 characters)
print("Text extracted from PDF:")
print(text[:10000])


Saving 21112048_Software Resume_2024-06-02_21_22_19.pdf to 21112048_Software Resume_2024-06-02_21_22_19.pdf
Text extracted from PDF:
 Area of Interest
Full-Stack Web Development, Application Development, Data Structures and Algorithms, Competitive Programming
 Education
Year 
Degree/Examination 
Institution/Board 
CGPA/
Percentage 
2024
B.T
ech. 3rd Year
Indian Institute of T
echnology, Roorkee
7.892
2021
Intermediate (Class XII)
Ebenezer Higher secondary School ,Bhagat Singh Nagar,
Near Deen Dayal Nagar, Gwalior
86.20 % 
2019
Matriculate (Class X)
Kendriya Vidyalaya No 2 AFS Maharajpur Gwalior MP
91.20 % 
 Internships
Advanced AI Trainer Software Development (Full-Stack Developers) | Invisible T
echnologies 
Spearheaded AI analytical capabilities enhancement, focusing on fact discernment and behavioral pattern recognition.
Leveraged Python, JavaScript, React, Node.js, Java, SQL, C++, PHP, CSS, and Kotlin to contribute to Full-Stack projects.
Evaluated AI performance and conducted stre

In [5]:
# Initialize the QA pipeline
qa_pipeline = create_qa_pipeline()

# Define the context (entire text in this case)
context = text

# Ask a question
question = input("Ask a question: ")

# Get the answer
answer = answer_question(question, context, qa_pipeline)
print("Answer:")
print(answer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Ask a question: Name the professor in the references section?
Answer:
Prasenjit Mondal


In [10]:
import streamlit as st

def main():
    st.title("PDF Question Answering System")

    # Upload PDF
    pdf_file = st.file_uploader("Upload PDF", type=["pdf"])

    if pdf_file is not None:
        # Extract text
        text = extract_text_from_pdf(pdf_file)
        st.write("Text extracted from PDF:")
        st.write(text[:1000])  # Display the first 1000 characters

        # Preprocess and get context
        context = preprocess_text(text)

        # Ask question
        question = st.text_input("Ask a question:")

        if st.button("Get Answer"):
            answer = answer_question(question, text, qa_pipeline)
            st.write("Answer:")
            st.write(answer)

if __name__ == '__main__':
    main()


In [ ]:
!streamlit run app.py